# Batch Slide label cropping

----
### Import dependancies

In [1]:
import pytesseract as pytess
import numpy as np 
import skimage 
import scipy 
# import aicsimageio
import os 
import tifffile as tf
from PIL import Image
import tkinter as tk 
import pandas as pd
import matplotlib.pyplot as plt
from tkinter import filedialog

In [2]:
# pytess.pytesseract.tesseract_cmd = r'C:/Users/rcorbyn/AppData/Local/Programs/Tesseract-OCR/tesseract.exe'
pytess.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

---- 
### Get all files

In [3]:
def get_files(im_folders):
    ''' '''

    # initalise 
    im_files = []
    #
    for root, folders, files in os.walk(im_folders):
        for file in files:
            if file[-4:] == '.png': 
                im_files.append(root + '/' +file)

    return(im_files)

----
### Get slide image

In [15]:
# # # Creates dialogue to ask directory
# # # Get the folder containing the image stack. 
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw() # Stops a second window opening
folder = filedialog.askdirectory(title = 'Select Stack file')

files = get_files(folder) 

# for i in range( len(files) ):
#     print(files[i])
print(folder)

N:/RCORBYN/User_Data/Current_Projects/20250703_Danis/Bcl-2


----
### Letter segmentation

In [16]:
# Initalise
results = []
files[0]
save_loc = folder + '/Labels/'
if os.path.exists(save_loc) == False: 
    os.makedirs(save_loc)

print(save_loc)

for file in files:
    #
    if file[-4:] == '.png':
        print(os.path.basename(file))
        # Open up the image
        rgba_image = Image.open( file)
        im_data = rgba_image.convert('RGB')
        # Reduce the image to the lowest 
        # number of dimensions. 
        im_data = np.array(im_data)
        #######
        # Perform edge detection
        edges = skimage.filters.sobel( np.sum(im_data, axis = 2) )
        
        #####
        # Threshold the image to remove most of the 
        # unimportant information from the image.
        
        # Find the threshold values
        threshold_val = skimage.filters.threshold_otsu(edges)
        edges[edges < threshold_val] = 0
        
        # Fill the binary masks of the letters
        fill_string = scipy.ndimage.binary_fill_holes(edges)
        
        # Invert the image and multiply by the letter
        # mask to get only the letters in the image. 
        invert = fill_string * skimage.util.invert( im_data[:, :, 0] )
        #####
        # Create a binary image from the inverted letters
        binary = np.array(invert)
        binary[binary>0] = 1
        
        # create labels from the binary image. 
        labels = skimage.measure.label(binary)
        
        # Get properties of the letters and remaining masks. 
        region_props = pd.DataFrame( skimage.measure.regionprops_table(
                        labels, properties = {'label', 'area', 
                        'axis_minor_length', 'axis_major_length'}) )
        
        # Create a mask of zeros
        filtered_image = np.zeros(im_data.shape)
        
        # Loop around all labels. 
        for i in range(region_props.shape[0]): 
            # Find the positon of mask i in the label image. 
            mask_pos = np.where(labels == region_props['label'].iloc[i])
            # Find the maximum and minimum height and width of the masks. 
            delta_x = np.abs( np.min(mask_pos[0]) - np.max(mask_pos[0]) ) 
            delta_y = np.abs( np.min(mask_pos[1]) - np.max(mask_pos[1]) ) 
            # If the mask is too long or high, remove 
            # Also remove if the area is too small. 
            if delta_x  <= 75 and delta_y <= 75 and region_props['area'].iloc[i] > 250: 
                # keep the mask. 
                filtered_image[mask_pos[0], mask_pos[1]] = skimage.util.invert(
                                            im_data[mask_pos[0], mask_pos[1], :] )
            
        filtered_image = np.array(filtered_image, dtype = 'uint8')
        
        filtered_image[filtered_image>150] = 255
        filtered_image[filtered_image<150] = 0

        tf.imwrite(save_loc + os.path.basename(file[0:-4]) + '_label.png', filtered_image)

N:/RCORBYN/User_Data/Current_Projects/20250703_Danis/Bcl-2/Labels/
labelled_R09_DT_060625_BCL-007.png
labelled_R09_DT_060625_BCL-006.png
labelled_R09_DT_060625_BCL-021.png
labelled_R09_DT_060625_BCL-002.png
labelled_R09_DT_060625_BCL-019.png
labelled_R09_DT_060625_BCL-016.png
labelled_R09_DT_060625_BCL-023.png
labelled_R09_DT_060625_BCL-022.png
labelled_R09_DT_060625_BCL-003.png
labelled_R09_DT_060625_BCL-027.png
labelled_R09_DT_060625_BCL-004.png
labelled_R09_DT_060625_BCL-025.png
labelled_R09_DT_060625_BCL-014.png
labelled_R09_DT_060625_BCL-015.png
labelled_R09_DT_060625_BCL-001.png
labelled_R09_DT_060625_BCL-018.png
labelled_R09_DT_060625_BCL-009.png
labelled_R09_DT_060625_BCL-008.png
labelled_R09_DT_060625_BCL-020.png
labelled_R09_DT_060625_BCL-028.png
labelled_R09_DT_060625_BCL.png
labelled_R09_DT_060625_BCL-011.png
labelled_R09_DT_060625_BCL-005.png
labelled_R09_DT_060625_BCL-024.png
labelled_R09_DT_060625_BCL-010.png
labelled_R09_DT_060625_BCL-012.png
labelled_R09_DT_060625_BCL-

In [10]:
save_loc + os.path.basename(file)

'//data.beatson.gla.ac.uk/data/RCORBYN/User_Data/Current_Projects/20250703_Danis/Bcl-2/work/Labels/labelled_R09_DT_060625_BCL-013.png'